In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EmployeeData").getOrCreate()
df = spark.read.csv("/FileStore/tables/employees-1.csv", header=True, inferSchema=True)
df.show(10)

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [0]:
df.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [0]:
from pyspark.sql.functions import col

df_updated = df.withColumn("SALARY", col("SALARY") + 30000)
df_updated.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK| 32600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK| 32600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST| 34400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 43000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP| 36000|            - |       201|           20|


##Coding: Filter out rows where salary < 50,000

In [0]:
df_filtered = df_updated.filter(df_updated.SALARY >= 50000)
df_filtered.show()

+-----------+----------+---------+-----+------------+---------+-------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+-----+------------+---------+-------+------+--------------+----------+-------------+
|        100|    Steven|     King|SKING|515.123.4567|17-JUN-03|AD_PRES| 54000|            - |        - |           90|
+-----------+----------+---------+-----+------------+---------+-------+------+--------------+----------+-------------+



##Coding: Read CSV, drop nulls, save as Delta table

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EmployeeData1").getOrCreate()
df1 = spark.read.csv("/FileStore/tables/employees-1.csv", header=True, inferSchema=True)
df1.show(10)

df_cleaned = df1.replace("-", None)
df_cleaned = df_cleaned.dropna()

df_cleaned.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("default.cleaned_employees1")

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


##Coding: Group by department and calculate avg salary

In [0]:
from pyspark.sql.functions import avg
df_grouped = df1.groupBy("DEPARTMENT_ID").agg(avg("SALARY").alias("AVG_SALARY"))
df_grouped.show()

+-------------+------------------+
|DEPARTMENT_ID|        AVG_SALARY|
+-------------+------------------+
|           20|            9500.0|
|           40|            6500.0|
|          100| 8601.333333333334|
|           10|            4400.0|
|           50|3721.7391304347825|
|           70|           10000.0|
|           90|19333.333333333332|
|           60|            5760.0|
|          110|           10154.0|
|           30|            4150.0|
+-------------+------------------+



##Coding: Remove duplicate rows based on email

In [0]:
df_deduplicated = df1.dropDuplicates(["EMAIL"])
df_deduplicated.show()

+-----------+-----------+-----------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID| FIRST_NAME|  LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+-----------+-----------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        121|       Adam|      Fripp|  AFRIPP|650.123.2234|10-APR-05|    ST_MAN|  8200|            - |       100|           50|
|        103|  Alexander|     Hunold| AHUNOLD|590.423.4567|03-JAN-06|   IT_PROG|  9000|            - |       102|           60|
|        115|  Alexander|       Khoo|   AKHOO|515.127.4562|18-MAY-03|  PU_CLERK|  3100|            - |       114|           30|
|        104|      Bruce|      Ernst|  BERNST|590.423.4568|21-MAY-07|   IT_PROG|  6000|            - |       103|           60|
|        105|      David|     Austin| DAUSTIN|590.423.4569|25-JUN-05|   IT_PROG|  4800|            - |  

##Short Answer: Explain lazy evaluation in Spark

Spark delays execution of transformations until an action is triggered.
It builds a logical execution plan (DAG) instead of running each step immediately.
This optimization helps reduce redundant computations and improves performance.
Transformations (e.g., filter(), map()) are lazy, meaning they don’t execute immediately.
Actions (e.g., show(), collect(), count()) trigger execution of all preceding transformations.

In [0]:
df_filtered = df.filter(df["SALARY"] > 10000)  # Transformation (Lazy, no execution yet)
df_filtered = df_filtered.select("EMPLOYEE_ID", "SALARY")  # Another transformation (Still no execution)
df_filtered.show()  # Action (Now Spark runs all transformations)

+-----------+------+
|EMPLOYEE_ID|SALARY|
+-----------+------+
|        201| 13000|
|        205| 12008|
|        100| 24000|
|        101| 17000|
|        102| 17000|
|        108| 12008|
|        114| 11000|
+-----------+------+

